# Deduper  
---  
## Part 1  
---  
### Define the Problem  
---  
-  _PCR duplicates arise during PCR amplification of original fragmented, adapter-ligated DNA. They become problematic when 2 copies of the same molecule get onto different primer lawns on an Illumina flowcell. PCR-duplicate removal (or tagging) is necessary to mitigate PCR bias, which is introduced when certain molecules ligate adapters more favorably and are disproportionately amplified. The greater numbers of these preferentially amplified molecules makes them more likely to contaminate neighboring lawns on the flowcell. Failure to remove these duplicates could limit your ability to draw conclusions from your data, as biological information may be confounded by PCR bias._  

### Write Examples  
---  
   -  Input SAM file:  
       -  **sample\_input.sam**  
           -  6 conditions for forward read:
               -  Initial UMI (non-duplicate)
               -  Same UMI, different chromosome
               -  Same UMI, different POS  
               -  Same UMI, different strand
               -  Same UMI, DUPLICATE w/ soft-clipping
               -  Ssme UMI, DUPLICATE w/o soft-clipping
           -  6 conditions for reverse read:
               -  Same as above
   -  Output SAM file:  
       -  **sample\_output.sam**  
           -  8 reads: 4 from forward, 4 from reverse.

### Develop Algorithm w/ Pseudocode  
---  

In [ ]:
#################
## PSEUDO-CODE ##
#################
# To run SAMtools on the command line from a Python script...
from subprocess import call  
# Import regular expressions.
import re

# run SAMtools sort on file(s).
call("samtools sort -l 0 -m maxMem -o <out.sam> -T <out.prefix> <input_file.sam>", shell=True)

# open the file produced from 'samtools sort' (above) as fin, open the output file for THIS program as fout.
# open the file of known barcodes as index.
with open("out.sam") as fin, with open("dedup_output.sam",'a') as fout, with open("STL96.txt") as index:
    # Create an empty set to hold real indexes.
    valid_IDs = set()
    # Add each index to valid_IDs.
    for i in index:
        i = i.strip()
        valid_IDs.add(i)
    # Create a counter for PCR-duplicates.
    PCR_dup = 0
    # Create a dictionary to keep track of UMI information.
    UMI_dict = {}
    # Keep track of total reads.
    tot_rds = 0
    # Iterate through each line of the input SAM file.
    for line in fin:
        # If the line is a header line...
        if re.match(r'^@[A-Z]{2}\s', line):
            # Store the line w/newline in the temporary string 'header'.
            header = (line + "\n")
            # Write the header to fout.
            fout.write(header)
            # Continue iterating thru 'fin'.
            continue
        else:
            # Increment the number of total reads.
            tot_rds += 1
            # Get the relevant information to check for PCR duplicates.
            parse_read = parseSAM(line)
            # Store info in separate variables.
            this_UMI = parse_read[0]
            this_chr = parse_read[1]
            this_pos = parse_read[2]
            this_CIGAR = parse_read[3]
            this_strand = parse_read[4]
            # If current UMI is in valid_IDs...
            if this_UMI in valid_IDs:
                # If current read HASN'T undergone soft-clipping...
                if this_pos == truePOS(this_pos,this_CIGAR):
                    # And current read is not in the dictionary.
                    if this_UMI not in UMI_dict:
                        # Add it to the dictionary.
                        UMI_dict[this_UMI] = (this_chr,this_pos,this_strand)
                        read = line + "\n"
                        # Write the line to fout.
                        fout.write(read)
                    # If the UMI is already in the dictionary, check for PCR duplicate...
                    elif:
                        if this_chr == UMI_dict[this_UMI][0] and this_pos == UMI_dict[this_UMI][1] and this_strand == UMI_dict[this_UMI[2]]:
                            # Same UMI, same chromosome, same position, and same strand... This is a PCR duplicate.
                            # Don't write the line to file. Continue through file.
                            # Increment PCR-duplicate counter.
                            PCR_dup += 1
                            continue
                    # If the UMI is in dictionary, but not a duplicate...
                    else:
                        read = line + "\n"
                        # Write the read to fout.
                        fout.write(read)
                # The current read HAS undergone soft-clipping...
                else:
                    # Correct the position.
                    this_pos = truePOS(this_pos,this_CIGAR)
                    # Check for PCR-duplicate.
                    if this_chr == UMI_dict[this_UMI][0] and this_pos == UMI_dict[this_UMI][1] and this_strand == UMI_dict[this_UMI[2]]:
                            # Same UMI, same chromosome, same position, and same strand... This is a PCR duplicate.
                            # Don't write the line to file. Continue through file.
                            # Increment PCR-duplicate counter.
                            PCR_dup += 1
                            continue
                    # The read is not a PCR duplicate, write to fout.
                    else:
                        read = line + "\n"
                        fout.write(read)
            # If the UMI isn't valid, DO NOT write the current read to the output file.    
            else:
                # Error correction on UMIs would occur in this block.
                continue
            
            
            
            


### Determine High-level functions  
---  

In [44]:
import re

def truePOS(POS,CIGAR):
    """
    description: truePOS checks if soft-clipping has occurred at the left-most mapped position on a read.
    If soft-clipping has occurred, truePOS performs a calculation to determine the INT value for the original 
    left-most-mapped position and returns this corrected pos. If soft-clipping has not occurred, truePOS returns
    the original INT value of POS.
    INPUT: POS = A STR representing the left-most-mapped position of the read.
    CIGAR = A STR representing the CIGAR-string for the read.
    OUTPUT: pos = A SC-corrected value for POS in reads where soft-clipping has occurred.
    POS = The original integer value for POS
    """
    # Make POS an INT for calculations.
    POS = int(POS)
    # Search for left-most soft-clipping.
    sc = re.search("([0-9]{1,2})S[0-9]{1,2}", CIGAR)
    # If it's found...
    if sc:
        # Set the difference to the INT value before the first S, for calculations.
        diff = int(sc.group(1))
        # Create the new corrected position, to check for PCR duplicates.
        pos = POS - diff
        # Return the corrected position.
        return pos
    # If there's no soft-clipping...
    else:
        # Return the original position
        return POS
    
def parseSAM(lineSAM):
    """
    description: parseSAM takes 1 line from a SAM file and returns the UMI, chromosome #, POS, CIGAR, and the strand.
    INPUT: A SAM file read-line
    OUTPUT: UMI,CHROM,POS,CIGAR,STRAND
    """
    umi = re.search(".*:([A,C,G,T]{8})\t", lineSAM)
    barcode = umi.group(1)
    line = lineSAM.split("\t")
    chromo = line[2]
    POS = line[3]
    CIGAR = line[5]
    flag = int(line[1])
    if((flag & 16) != 16):
        strand = "+"
    else:
        strand = "-"
    return barcode,chromo,POS,CIGAR,strand

In [24]:
# Test truePOS()
assert(truePOS("500","10S90M1S")) == 490
assert(truePOS("500","90M11S")) == 500
assert(truePOS("500","1S98M2S")) == 499
assert(truePOS("1","101M")) == 1

In [46]:
# Test parseSAM()
import re
# Use a known test file on local computer to test parseSAM().
with open("testset3.sam") as f:
    header = ""
    LN = 0
    for line in f:
        if re.match(r'^@[A-Z]{2}\s', line):
            continue
        else:
            LN += 1
            if LN < 9:
                stuff = parseSAM(line)
                assert(len(stuff[0])) == 8
                assert(stuff[1]) == "1"
                assert(stuff[3]) == "71M"
                assert(stuff[4]) == "+"